Calculate Peak over Standard Deviation

In [ ]:
from electroviz.core.experiment import Experiment
from electroviz.utils.parse_units import read_summaries
from electroviz.utils.cross_corr import cross_corr
import numpy as np
import pandas as pd

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("depth")
pop1 = exp.populations[1]
pop1 = pop1.remove(pop1.units["total_spikes"] < 100)
pop1_sort = pop1.sort("depth")

# ctx_idx_0, ctx_idx_1 = read_summaries("G:/Shared drives/Neuropixels/Figures/RG001/UnitSummary/CTX")
# lgn_idx = read_summaries("G:/Shared drives/Neuropixels/Figures/RG001/UnitSummary/LGN")[0]

# xcorr_df = pd.DataFrame()

# for ctx in ctx_idx_0:
#     for lgn in lgn_idx:

#         ctx_unit = pop0_sort[ctx]
#         lgn_unit = pop0_sort[lgn]

#         xcorr_raw, xcorr_filt = cross_corr((ctx_unit, lgn_unit))

#         mask = np.array([False] * xcorr_filt.size)
#         mask[52:57] = True, True, True, True, True

#         peak = np.max(xcorr_filt[mask])
#         mean = np.mean(xcorr_filt[mask == False])
#         std = np.std(xcorr_filt[mask == False])
#         ratio = (peak - mean) / std

#         df = pd.DataFrame({"ctx_probe": [1], 
#                            "ctx_idx" : [ctx], 
#                            "lgn_probe": [1], 
#                            "lgn_idx" : [lgn], 
#                            "peak" : [np.around(peak, decimals=2)], 
#                            "mean" : [np.around(mean, decimals=2)], 
#                            "std" : [np.around(std, decimals=2)], 
#                            "ratio" : [np.around(ratio, decimals=2)], 
#                            })

#         xcorr_df = pd.concat((xcorr_df, df))

# for ctx in ctx_idx_1:
#     for lgn in lgn_idx:

#         ctx_unit = pop1_sort[ctx]
#         lgn_unit = pop0_sort[lgn]

#         xcorr_raw, xcorr_filt = cross_corr((ctx_unit, lgn_unit))

#         mask = np.array([False] * xcorr_filt.size)
#         mask[52:57] = True, True, True, True, True

#         peak = np.max(xcorr_filt[mask])
#         mean = np.mean(xcorr_filt[mask == False])
#         std = np.std(xcorr_filt[mask == False])
#         ratio = (peak - mean) / std

#         df = pd.DataFrame({"ctx_probe": [2], 
#                            "ctx_idx" : [ctx], 
#                            "lgn_probe": [1], 
#                            "lgn_idx" : [lgn], 
#                            "peak" : [np.around(peak, decimals=2)], 
#                            "mean" : [np.around(mean, decimals=2)], 
#                            "std" : [np.around(std, decimals=2)], 
#                            "ratio" : [np.around(ratio, decimals=2)], 
#                            })

#         xcorr_df = pd.concat((xcorr_df, df))

Plot PairSummaries for Sharp Peaks

In [ ]:
import pandas as pd
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import PairSummary
import matplotlib.pyplot as plt

xcorr_df = pd.read_csv("G:/Shared drives/Neuropixels/Data/RG001/xcorr_df.csv")
df_iter = xcorr_df.sort_values("ratio")[::-1][31:100][["ctx_probe", "ctx_idx", "lgn_probe", "lgn_idx"]]

for ctx_probe, ctx_idx, lgn_probe, lgn_idx in df_iter.itertuples(index=False):

    if ctx_probe == 1:
        ctx_unit = pop0_sort[ctx_idx]
    else:
        ctx_unit = pop1_sort[ctx_idx]
    
    lgn_unit = pop0_sort[lgn_idx]

    ctx_kernels = [SparseNoiseKernel(ctx_unit, stimuli[0]), 
                   SparseNoiseKernel(ctx_unit, stimuli[1]), 
                   StaticGratingsKernel(ctx_unit, stimuli[2]), 
                   StaticGratingsKernel(ctx_unit, stimuli[3])]

    lgn_kernels = [SparseNoiseKernel(lgn_unit, stimuli[0]), 
                   SparseNoiseKernel(lgn_unit, stimuli[1]), 
                   StaticGratingsKernel(lgn_unit, stimuli[2]), 
                   StaticGratingsKernel(lgn_unit, stimuli[3])]

    file_path = "G:/Shared drives/Neuropixels/Figures/RG001/PairSummary/"
    file_name = "Probe{0}_Unit{1}_Probe1_Unit{2}.png".format(str(ctx_probe), str(ctx_idx), str(lgn_idx))

    PairSummary((ctx_unit, lgn_unit), stimuli[:4], (ctx_kernels, lgn_kernels), save_path=file_path+file_name)

    plt.close("all")